In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl
from tqdm import tqdm

from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim
import os


### truncate colormaps

In [ ]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    '''
    https://stackoverflow.com/a/18926541
    '''
    if isinstance(cmap, str):
        cmap = plt.get_cmap(cmap)
    new_cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap


cmap_Blues = truncate_colormap('Blues', 0.4, 1)
cmap_PuBuGn = truncate_colormap('PuBuGn', 0.2, 1)
cmap_Greens = truncate_colormap('Greens', 0.4, 1)
cmap_Oranges = truncate_colormap('Oranges', 0.4, 1)

In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10

BUILD_ARRAYS = False
SAVEFIG = False
TOL=0.01
MAXBADS=5

NVARS = 3
# NVARS = 6

NPSRS = 40
NSKIES = 25
# NSKIES = 15

# Vary Parameter


In [ ]:
def get_data(
        target, nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz'     
):

    if path == '/Users/emigardiner/GWs/holodeck/output/anatomy_09B':
        load_data_from_file = path+f'/{target}_v{nvars}_r{nreals}_s{nskies}_shape{str(shape)}.npz' 
    else:
        load_data_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/data_params.npz' 

    if os.path.exists(load_data_from_file) is False:
        err = f"load data file '{load_data_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    # if os.path.exists(load_dets_from_file) is False:
    #     err = f"load dets file '{load_dets_from_file}' does not exist, you need to construct it."
    #     raise Exception(err)
    file = np.load(load_data_from_file, allow_pickle=True)
    data = file['data']
    params = file['params']
    file.close()

    # file = np.load(load_dets_from_file, allow_pickle=True)
    # dsdat = file['dsdat']
    # file.close()

    return data, params

# Save arrays

In [ ]:
targets = ['hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10',  'mmb_mamp_log10', 
            'mmb_scatter_dex', 'hard_gamma_inner']
path='/Users/emigardiner/GWs/holodeck/output/anatomy_redz'
NVARS = 21
NSKIES = 50
NREALS = 500

for TARGET in tqdm(targets):
    parvars = [0,5,10,15,20] 
    yy_ss = []
    yy_bg = []
    data, params = get_data(TARGET, nvars=NVARS, nskies=NSKIES, nreals=NREALS,
                            path=path)
    xx=data[0]['fobs_cents']*YR
    for vv, var in enumerate(parvars):
        hc_ss = data[var]['hc_ss']
        hc_bg = data[var]['hc_bg']

        sspar = data[var]['sspar']
        bgpar = data[var]['bgpar']

        sspar = sings.all_sspars(fobs_cents, sspar)
        bgpar = bgpar*sings.par_units[:,np.newaxis,np.newaxis]
        sspar = sspar*sings.par_units[:,np.newaxis,np.newaxis,np.newaxis]

    # parameters to plot
        _yy_ss = [hc_ss[...,0], sspar[0,...,0], #sspar[1,...,0], # sspar[2,],  # strain, mass, mass ratio,
                sspar[4,...,0]] # final comoving distance, single loudest only

        _yy_bg = [hc_bg, bgpar[0], #bgpar[1],  # strain, mass, mass ratio, initial redshift, final com distance
                bgpar[4],]
        yy_ss.append(_yy_ss)
        yy_bg.append(_yy_bg)

    save_name = path+f'/figdata/hcpar_arrays_{TARGET}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz'
    np.savez(save_name, xx=xx, yy_ss=yy_ss, yy_bg=yy_bg)

# Load arrays

In [ ]:
def load_hcpar_arrays(target, 
    path='/Users/emigardiner/GWs/holodeck/output/anatomy_redz'):
    
    load_name = path+f'/figdata/hcpar_arrays_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz'
    file = np.load(load_name)
    xx = file['xx']
    yy_ss = file['yy_ss']
    yy_bg = file['yy_bg']
    file.close()
    return xx, yy_ss, yy_bg

# Draw Functions

In [ ]:
def draw_95ci(ax, xx, yy_ss, yy_bg, ii, colors,
                parvars=[0,5,10,15,20]):

    handles=[]
    for vv, var in enumerate(parvars): 
        if var is not 5 and var is not 15:
            hh, = ax.plot(xx, np.median(yy_bg[vv][ii], axis=-1), color=colors[var], lw=1, linestyle='--')
            # for aa, nn in enumerate(idx):
            #     ax.plot(xx, yy_bg[ii][:,nn], linestyle='-', alpha=0.75, color=colors[aa])  
            handles.append(hh)

    for vv, var in enumerate(parvars):
        if var is not 5 and var is not 15:
        # Plot the loudest single sources confidence intervals
            for pp in [95,]:
                percs = pp / 2
                percs = [50 - percs, 50 + percs]
                ax.fill_between(xx, *np.percentile(yy_ss[vv][ii], percs, axis=-1), alpha=0.15, color=colors[var])
            
    return handles
        


In [ ]:
def draw_lims(ax, xx, yy_ss, yy_bg, ii, colors):

    # Plot the median 
    handles=[]
    for vv, var in enumerate(parvars): 
        hh, = ax.plot(xx, np.median(yy_bg[vv][ii], axis=-1), color=colors[var], lw=1, linestyle='--')
        # for aa, nn in enumerate(idx):
        #     ax.plot(xx, yy_bg[ii][:,nn], linestyle='-', alpha=0.75, color=colors[aa])  
        handles.append(hh)


    for vv, var in enumerate(parvars):
        # for aa, nn in enumerate(idx):
            # for ll in range(3):
            # edgecolor = 'k' if ll==0 else None
        ymed = np.median(yy_ss[vv][ii], axis=-1)
        ymax = np.max(yy_ss[vv][ii], axis=-1)
        ymin = np.min(yy_ss[vv][ii], axis=-1)
        if ii == 2:
            ax.errorbar(xx, ymin, yerr=(ymin-ymin, ymed-ymin), color=colors[var], alpha=0.15, 
                    capsize=0.5, lolims=True, marker='o', markersize=1, linestyle='')
        else:
            ax.errorbar(xx, ymax, yerr=(ymax-ymed, ymax-ymax), color=colors[var], alpha=0.15, 
                    capsize=0.5, uplims=True, marker='o', markersize=1, linestyle='')
            # ax.scatter(xx, ymed, marker='o', color=colors[vv], alpha=0.5, s=20)
            
    for vv, var in enumerate(parvars):
        # Plot the loudest single sources confidence intervals
        for pp in [68,]:
            percs = pp / 2
            percs = [50 - percs, 50 + percs]
            ax.fill_between(xx, *np.percentile(yy_ss[vv][ii], percs, axis=-1), alpha=0.15, color=colors[var])
    
    return handles 

# 6 Panel Plot

In [ ]:
targets = ['gsmf_phi0', 'gsmf_mchar0_log10', 
            'mmb_mamp_log10', 'mmb_scatter_dex', 
            'hard_time', 'hard_gamma_inner']
color_maps = [
    cmap_Greens(np.linspace(0, 1, NVARS)), cmap_Greens(np.linspace(0, 1, NVARS)),
    cmap_Oranges(np.linspace(0, 1, NVARS)), cmap_Oranges(np.linspace(0, 1, NVARS)),
    cmap_Blues(np.linspace(0, 1, NVARS)), cmap_Blues(np.linspace(0, 1, NVARS)),
]

ylim = [
    (2e-19, 2e-13), # hc
    (6.5e5, 8.5e10), # mass in Msun
    (4e1, 7e3)
]

ylabels = ['$h_c$', '$M\ [\mathrm{M}_\odot]$', '$d_c\ [\mathrm{Mpc}]$',]
titles = ['$\psi_0$', '$m_{\psi,0}$', '$\mu$', '$\epsilon_\mu$', 
            '$\\tau_\mathrm{hard}$', '$\\nu_\mathrm{inner}$']

fig, axs = plot.figax(ncols=6, nrows=3, sharex=True, figsize=(12,4),)
fig.text(0.5,0.0, plot.LABEL_GW_FREQUENCY_YR, ha='center')
for ii, ax in enumerate(axs[:,0]):
    ax.set_ylabel(ylabels[ii])

for cc in range(6):
    xx, yy_ss, yy_bg = load_hcpar_arrays(targets[cc])
    for rr, ax in enumerate(axs[:,cc]):
        handles = draw_95ci(ax, xx, yy_ss, yy_bg, ii=rr, colors=color_maps[cc])
        ax.set_ylim(ylim[rr])
        if cc>0:
            ax.set_yticklabels([])
        if rr==0:
            # ax.text(0.98, 0.98, titles[cc], ha='right', va='top', transform=ax.transAxes)
            ax.text(0.0, 0.0, plot.PARAM_KEYS[targets[cc]], ha='left', va='bottom', 
                    transform=ax.transAxes, fontsize=12)

            # ax.sharey(axs[rr,0])
plt.subplots_adjust(hspace=0, wspace=0)


save_path = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/new_spectra'
save_name = save_path+'/95ci_var3_3x6.png'
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
targets = ['gsmf_phi0', 'gsmf_mchar0_log10', 
            'mmb_mamp_log10', 'mmb_scatter_dex', 
            'hard_time', 'hard_gamma_inner']
color_maps = [
    cmap_Greens(np.linspace(0, 1, NVARS)), cmap_Greens(np.linspace(0, 1, NVARS)),
    cmap_Oranges(np.linspace(0, 1, NVARS)), cmap_Oranges(np.linspace(0, 1, NVARS)),
    cmap_Blues(np.linspace(0, 1, NVARS)), cmap_Blues(np.linspace(0, 1, NVARS)),
]

ylim = [
    (2e-19, 3e-13), # hc
    (6.5e5, 2e11), # mass in Msun
    (3e1, 5e3)
]

ylabels = ['$h_c$', '$M\ [\mathrm{M}_\odot]$', '$d_c\ [\mathrm{Mpc}]$',]
titles = ['$\psi_0$', '$m_{\psi,0}$', '$\mu$', '$\epsilon_\mu$', 
            '$\\tau_\mathrm{hard}$', '$\\nu_\mathrm{inner}$']

fig, axs = plot.figax(ncols=6, nrows=3, sharex=True, figsize=(12,5),)
fig.text(0.5,0.0, plot.LABEL_GW_FREQUENCY_YR, ha='center')
for ii, ax in enumerate(axs[:,0]):
    ax.set_ylabel(ylabels[ii])

for cc in range(6):
    xx, yy_ss, yy_bg = load_hcpar_arrays(targets[cc])
    for rr, ax in enumerate(axs[:,cc]):
        handles = draw_lims(ax, xx, yy_ss, yy_bg, ii=rr, colors=color_maps[cc])
        ax.set_ylim(ylim[rr])
        if cc>0:
            ax.set_yticklabels([])
        if rr==0:
            # ax.text(0.98, 0.98, titles[cc], ha='right', va='top', transform=ax.transAxes)
            ax.text(0.0, 0.0, plot.PARAM_KEYS[targets[cc]], ha='left', va='bottom', 
                    transform=ax.transAxes, fontsize=12)

            # ax.sharey(axs[rr,0])
plt.subplots_adjust(hspace=0, wspace=0)


save_path = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/new_spectra'
save_name = save_path+'/95lims_var3_3x6.png'
fig.savefig(save_name, dpi=300, bbox_inches='tight')

# 2 x 9

In [ ]:
targets = ['gsmf_phi0', 'gsmf_mchar0_log10', 
            'mmb_mamp_log10', 'mmb_scatter_dex', 
            'hard_time', 'hard_gamma_inner']
color_maps = [
    cmap_Greens(np.linspace(0, 1, NVARS)), cmap_Greens(np.linspace(0, 1, NVARS)),
    cmap_Oranges(np.linspace(0, 1, NVARS)), cmap_Oranges(np.linspace(0, 1, NVARS)),
    cmap_Blues(np.linspace(0, 1, NVARS)), cmap_Blues(np.linspace(0, 1, NVARS)),
]

ylim = [
    (2e-19, 2e-13), # hc
    (6.5e5, 8.5e10), # mass in Msun
    (4e1, 7e3)
]

ylabels = ['$h_c$', '$M\ [\mathrm{M}_\odot]$', '$d_c\ [\mathrm{Mpc}]$',]

fig, axs = plot.figax_single(ncols=2, nrows=9, sharex=True, height=10,)
fig.text(0.5,0.1, plot.LABEL_GW_FREQUENCY_YR, ha='center')
# for ii, ax in enumerate(axs[:,0]):
#     ax.set_ylabel(ylabels[ii])

for cc in range(6):
    col = cc%2 # 0 or 1
    xx, yy_ss, yy_bg = load_hcpar_arrays(targets[cc])
    for rr in range(3):
        row = rr + 3*int(cc/2)
        print(f"{col=}, {row=}")
        ax = axs[row, col]
        handles = draw_95ci(ax, xx, yy_ss, yy_bg, ii=rr, colors=color_maps[cc])
        ax.set_ylim(ylim[rr])
        if col==0:
            ax.set_ylabel(ylabels[row%3])
        else:
            ax.set_yticklabels([])
            # ax.sharey(axs[rr,0])



plt.subplots_adjust(hspace=0, wspace=0)

# Old Versions

In [ ]:
ylabels = ['Char. Strain', 'Mass [M$_\odot$]', #'Mass Ratio', 
        'Distance [Mpc]',]

fig, axs = plot.figax_single(nrows=3, ncols=1, sharex=True, height=10)
xx = fobs_cents*YR
nsamp = 5

colors = cmap_Blues(np.linspace(0, 1, NVARS))

for ii, ax in enumerate(axs.flatten()):
    # Plot the median 
    handles=[]
    for vv, var in enumerate(parvars): 
        hh, = ax.plot(xx, np.median(yy_bg[vv][ii], axis=-1), color=colors[var])
        # for aa, nn in enumerate(idx):
        #     ax.plot(xx, yy_bg[ii][:,nn], linestyle='-', alpha=0.75, color=colors[aa])  
        handles.append(hh)

    for vv, var in enumerate(parvars):
        # for aa, nn in enumerate(idx):
            # for ll in range(3):
            # edgecolor = 'k' if ll==0 else None
        ymed = np.median(yy_ss[vv][ii], axis=-1)
        ymax = np.max(yy_ss[vv][ii], axis=-1)-ymed
        ymin = ymed - np.min(yy_ss[vv][ii], axis=-1)
        ax.errorbar(xx, ymed, yerr=(ymin, ymax), color=colors[var], alpha=0.5, 
                capsize=3, marker=None)
        ax.scatter(xx, ymed, marker='o', color=colors[vv], alpha=0.8, s=20)


    for vv, var in enumerate(parvars):
        # Plot the confidence intervals
        for pp in [95]:
            percs = pp / 2
            percs = [50 - percs, 50 + percs]
            ax.fill_between(xx, *np.percentile(yy_bg[vv][ii], percs, axis=-1), alpha=0.25, color=colors[var])


    # label axes
    # if ii>=3:
    ax.set_ylabel(ylabels[ii])

axs[-1].set_xlabel(plot.LABEL_GW_FREQUENCY_YR)
plt.subplots_adjust(wspace=0, hspace=0)
plot._twin_hz(axs[0], nano=True)

# fig.tight_layout()
labels = []
for vv, var in enumerate(parvars):
    labels.append(f"{params[var][TARGET]:.2f}")

fig.legend(handles=handles, labels=labels, bbox_to_anchor=(0.5,0.02), loc='lower center', ncols=len(parvars), title='$\\tau_\mathrm{hard}$')

## fave hard_time

In [ ]:
ylabels = ['Char. Strain', 'Mass [M$_\odot$]',# 'Mass Ratio', 
        'Distance [Mpc]',]

fig, axs = plot.figax_single(nrows=3, ncols=1, sharex=True, height=10)
xx = fobs_cents*YR
nsamp = 5

colors = cmap_Blues(np.linspace(0, 1, NVARS))
# colors = cmap_PuBuGn(np.linspace(0, 1, NVARS))

for ii, ax in enumerate(axs.flatten()):
    # Plot the median 
    handles=[]
    for vv, var in enumerate(parvars): 
        hh, = ax.plot(xx, np.median(yy_bg[vv][ii], axis=-1), color=colors[var], lw=3)
        # for aa, nn in enumerate(idx):
        #     ax.plot(xx, yy_bg[ii][:,nn], linestyle='-', alpha=0.75, color=colors[aa])  
        handles.append(hh)


    for vv, var in enumerate(parvars):
        # Plot the loudest single sources confidence intervals
        for pp in [95,]:
            percs = pp / 2
            percs = [50 - percs, 50 + percs]
            ax.fill_between(xx, *np.percentile(yy_ss[vv][ii], percs, axis=-1), alpha=0.25, color=colors[var])


    # label axes
    # if ii>=3:
    ax.set_ylabel(ylabels[ii])

axs[-1].set_xlabel(plot.LABEL_GW_FREQUENCY_YR)
plt.subplots_adjust(wspace=0, hspace=0)
plot._twin_hz(axs[0], nano=True)

# fig.tight_layout()
labels = []
for vv, var in enumerate(parvars):
    labels.append(f"{params[var][TARGET]:.2f}")

fig.legend(handles=handles, labels=labels, bbox_to_anchor=(0.5,0.02), loc='lower center', ncols=len(parvars), title='$\\tau_\mathrm{hard}$')
# fig.savefig(f'/Users/emigardiner/GWs/holodeck/output/figures/bigplots/charstrain_pars_vs_freq_var5_{TARGET}.png',
            # dpi=300)

Separate char strains

In [ ]:
parvars = [0,5,10,15,20]
hc_ss_all = []
hc_bg_all = []
text = []

data, params = get_data(TARGET, nvars=21, nreals=500, nskies=100, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
for vv, var in enumerate(parvars):    
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']
    hc_ss_all.append(hc_ss)
    hc_bg_all.append(hc_bg)
    text.append(params[var][TARGET])

# GSMF_Phi0, 5vars

In [ ]:
TARGET = 'gsmf_phi0'
NVARS = 21
NREALS = 500

parvars = [0,5,10,15,20]
yy_ss = []
yy_bg = []
data, params = get_data(TARGET, nvars=NVARS, nreals=NREALS,
                        path='/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
print(data.shape)
for vv, var in enumerate(parvars):
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']

    sspar = data[var]['sspar']
    bgpar = data[var]['bgpar']

    sspar = sings.all_sspars(fobs_cents, sspar)
    bgpar = bgpar*sings.par_units[:,np.newaxis,np.newaxis]
    sspar = sspar*sings.par_units[:,np.newaxis,np.newaxis,np.newaxis]


   # parameters to plot
    _yy_ss = [hc_ss[...,0], sspar[0,...,0], #sspar[1,...,0], # sspar[2,],  # strain, mass, mass ratio,
            sspar[4,...,0]] # final comoving distance, single loudest only

    _yy_bg = [hc_bg, bgpar[0], #bgpar[1],  # strain, mass, mass ratio, initial redshift, final com distance
            bgpar[4],]
    yy_ss.append(_yy_ss)
    yy_bg.append(_yy_bg)

In [ ]:
ylabels = ['Char. Strain', 'Mass [M$_\odot$]',# 'Mass Ratio', 
        'Distance [Mpc]',]

fig, axs = plot.figax_single(nrows=3, ncols=1, sharex=True, height=10)
xx = fobs_cents*YR
nsamp = 5

colors = cmap_Greens(np.linspace(0, 1, NVARS))
# colors = cmap_PuBuGn(np.linspace(0, 1, NVARS))

for ii, ax in enumerate(axs.flatten()):
    # Plot the median 
    handles=[]
    for vv, var in enumerate(parvars): 
        hh, = ax.plot(xx, np.median(yy_bg[vv][ii], axis=-1), color=colors[var], lw=2, linestyle='-')
        # for aa, nn in enumerate(idx):
        #     ax.plot(xx, yy_bg[ii][:,nn], linestyle='-', alpha=0.75, color=colors[aa])  
        handles.append(hh)

    for vv, var in enumerate(parvars):
        # Plot the loudest single sources confidence intervals
        for pp in [95,]:
            percs = pp / 2
            percs = [50 - percs, 50 + percs]
            ax.fill_between(xx, *np.percentile(yy_ss[vv][ii], percs, axis=-1), alpha=0.15, color=colors[var])
    
    for vv, var in enumerate(parvars):
        # for aa, nn in enumerate(idx):
            # for ll in range(3):
            # edgecolor = 'k' if ll==0 else None
        ymed = np.median(yy_ss[vv][ii], axis=-1)
        # ymax = np.max(yy_ss[vv][ii], axis=-1)-ymed
        # ymin = ymed - np.min(yy_ss[vv][ii], axis=-1)
        # ax.errorbar(xx, ymed, yerr=(ymin, ymax), color=colors[var], alpha=0.5, 
        #         capsize=3, marker=None)
        ax.scatter(xx, ymed, marker='o', color=colors[var], alpha=0.8, s=20)


    # label axes
    # if ii>=3:
    ax.set_ylabel(ylabels[ii])

axs[-1].set_xlabel(plot.LABEL_GW_FREQUENCY_YR)
plt.subplots_adjust(wspace=0, hspace=0)
plot._twin_hz(axs[0], nano=True)

# fig.tight_layout()
labels = []
for vv, var in enumerate(parvars):
    labels.append(f"{params[var][TARGET]:.2f}")

fig.legend(handles=handles, labels=labels, bbox_to_anchor=(0.5,0.02), loc='lower center', ncols=len(parvars), title=plot.PARAM_KEYS[TARGET])
fig.savefig(f'/Users/emigardiner/GWs/holodeck/output/figures/bigplots/hcprop_var/hcprop_vs_freq_var5_{TARGET}_midpoints.png',
            dpi=300)

In [ ]:
ylabels = ['Char. Strain', 'Mass [M$_\odot$]',# 'Mass Ratio', 
        'Distance [Mpc]',]

fig, axs = plot.figax_single(nrows=3, ncols=1, sharex=True, height=10)
xx = fobs_cents*YR
nsamp = 5

colors = cmap_Greens(np.linspace(0, 1, NVARS))
# colors = cmap_PuBuGn(np.linspace(0, 1, NVARS))

for ii, ax in enumerate(axs.flatten()):
    # Plot the median 
    handles=[]
    for vv, var in enumerate(parvars): 
        hh, = ax.plot(xx, np.median(yy_bg[vv][ii], axis=-1), color=colors[var], lw=2, linestyle='-')
        # for aa, nn in enumerate(idx):
        #     ax.plot(xx, yy_bg[ii][:,nn], linestyle='-', alpha=0.75, color=colors[aa])  
        handles.append(hh)

    for vv, var in enumerate(parvars):
        # Plot the loudest single sources confidence intervals
        for pp in [68,]:
            percs = pp / 2
            percs = [50 - percs, 50 + percs]
            ax.fill_between(xx, *np.percentile(yy_ss[vv][ii], percs, axis=-1), alpha=0.15, color=colors[var])
    
    for vv, var in enumerate(parvars):
        # for aa, nn in enumerate(idx):
            # for ll in range(3):
            # edgecolor = 'k' if ll==0 else None
        ymed = np.median(yy_ss[vv][ii], axis=-1)
        ymax = np.max(yy_ss[vv][ii], axis=-1)
        ymin = np.min(yy_ss[vv][ii], axis=-1)
        # ax.errorbar(xx, ymed, yerr=(ymin, ymax), color=colors[var], alpha=0.5, 
        #         capsize=3, marker=None)
        ax.scatter(xx, ymax, marker='v', color=colors[var], alpha=0.8, s=20)

        ax.scatter(xx, ymin, marker='^', color=colors[var], alpha=0.8, s=20)


    # label axes
    # if ii>=3:
    ax.set_ylabel(ylabels[ii])

axs[-1].set_xlabel(plot.LABEL_GW_FREQUENCY_YR)
plt.subplots_adjust(wspace=0, hspace=0)
plot._twin_hz(axs[0], nano=True)

# fig.tight_layout()
labels = []
for vv, var in enumerate(parvars):
    labels.append(f"{params[var][TARGET]:.2f}")

fig.legend(handles=handles, labels=labels, bbox_to_anchor=(0.5,0.02), loc='lower center', ncols=len(parvars), title=plot.PARAM_KEYS[TARGET])
fig.savefig(f'/Users/emigardiner/GWs/holodeck/output/figures/bigplots/hcprop_var/hcprop_vs_freq_var5_{TARGET}.png',
            dpi=300)

In [ ]:
def plot_95ci(xx, yy_ss, yy_bg, colors):
    ylabels = ['Char. Strain', 'Mass [M$_\odot$]',# 'Mass Ratio', 
            'Distance [Mpc]',]

    fig, axs = plot.figax_single(nrows=3, ncols=1, sharex=True, height=10)
    xx = fobs_cents*YR
    nsamp = 5

    # colors = cmap_Greens(np.linspace(0, 1, NVARS))
    # colors = cmap_PuBuGn(np.linspace(0, 1, NVARS))

    for ii, ax in enumerate(axs.flatten()):
        # Plot the median 
        handles=[]
        for vv, var in enumerate(parvars): 
            hh, = ax.plot(xx, np.median(yy_bg[vv][ii], axis=-1), color=colors[var], lw=3, linestyle='--')
            # for aa, nn in enumerate(idx):
            #     ax.plot(xx, yy_bg[ii][:,nn], linestyle='-', alpha=0.75, color=colors[aa])  
            handles.append(hh)

        for vv, var in enumerate(parvars):
            # Plot the loudest single sources confidence intervals
            for pp in [95,]:
                percs = pp / 2
                percs = [50 - percs, 50 + percs]
                ax.fill_between(xx, *np.percentile(yy_ss[vv][ii], percs, axis=-1), alpha=0.15, color=colors[var])
        

        # label axes
        # if ii>=3:
        ax.set_ylabel(ylabels[ii])

    axs[-1].set_xlabel(plot.LABEL_GW_FREQUENCY_YR)
    plt.subplots_adjust(wspace=0, hspace=0)
    plot._twin_hz(axs[0], nano=True)

    # fig.tight_layout()
    labels = []
    for vv, var in enumerate(parvars):
        labels.append(f"{params[var][TARGET]:.2f}")

    fig.legend(handles=handles, labels=labels, bbox_to_anchor=(0.5,0.02), loc='lower center', ncols=len(parvars), title=plot.PARAM_KEYS[TARGET])
    fig.savefig(f'/Users/emigardiner/GWs/holodeck/output/figures/bigplots/hcprop_var/hcprop_vs_freq_var5_{TARGET}_95ci.png',
                dpi=300)
    return fig
fig = plot_95ci(xx, yy_ss, yy_bg, colors = cmap_Greens(np.linspace(0, 1, NVARS)))

In [ ]:
def plot_errbars(xx, yy_ss, yy_bg, colors):
    ylabels = ['Char. Strain', 'Mass [M$_\odot$]', #'Mass Ratio', 
            'Distance [Mpc]',]

    fig, axs = plot.figax_single(nrows=3, ncols=1, sharex=True, height=10)
    xx = fobs_cents*YR
    nsamp = 5

    # colors = cmap_Greens(np.linspace(0, 1, NVARS))

    for ii, ax in enumerate(axs.flatten()):
        # Plot the median 
        handles=[]
        for vv, var in enumerate(parvars): 
            hh, = ax.plot(xx, np.median(yy_bg[vv][ii], axis=-1), color=colors[var])
            # for aa, nn in enumerate(idx):
            #     ax.plot(xx, yy_bg[ii][:,nn], linestyle='-', alpha=0.75, color=colors[aa])  
            handles.append(hh)

        for vv, var in enumerate(parvars):
            # for aa, nn in enumerate(idx):
                # for ll in range(3):
                # edgecolor = 'k' if ll==0 else None
            ymed = np.median(yy_ss[vv][ii], axis=-1)
            ymax = np.max(yy_ss[vv][ii], axis=-1)-ymed
            ymin = ymed - np.min(yy_ss[vv][ii], axis=-1)
            ax.errorbar(xx, ymed, yerr=(ymin, ymax), color=colors[var], alpha=0.5, 
                    capsize=3, marker=None)
            ax.scatter(xx, ymed, marker='o', color=colors[vv], alpha=0.8, s=20)


        for vv, var in enumerate(parvars):
            # Plot the confidence intervals
            for pp in [95]:
                percs = pp / 2
                percs = [50 - percs, 50 + percs]
                ax.fill_between(xx, *np.percentile(yy_bg[vv][ii], percs, axis=-1), alpha=0.25, color=colors[var])


        # label axes
        # if ii>=3:
        ax.set_ylabel(ylabels[ii])

    axs[-1].set_xlabel(plot.LABEL_GW_FREQUENCY_YR)
    plt.subplots_adjust(wspace=0, hspace=0)
    plot._twin_hz(axs[0], nano=True)

    # fig.tight_layout()
    labels = []
    for vv, var in enumerate(parvars):
        labels.append(f"{params[var][TARGET]:.2f}")

    fig.legend(handles=handles, labels=labels, bbox_to_anchor=(0.5,0.02), loc='lower center', ncols=len(parvars), title=plot.PARAM_KEYS[TARGET])
    fig.savefig(f'/Users/emigardiner/GWs/holodeck/output/figures/bigplots/hcprop_var/hcprop_vs_freq_var5_{TARGET}_errbars.png',
                dpi=300)
    return fig
fig = plot_errbars(xx, yy_ss, yy_bg, colors = cmap_Greens(np.linspace(0, 1, NVARS)))

In [ ]:
def plot_uplims(xx, yy_ss, yy_bg, colors):
    ylabels = ['Char. Strain', 'Mass [M$_\odot$]',# 'Mass Ratio', 
            'Distance [Mpc]',]

    fig, axs = plot.figax_single(nrows=3, ncols=1, sharex=True, height=10)
    xx = fobs_cents*YR
    nsamp = 5

    # colors = cmap_Greens(np.linspace(0, 1, NVARS))
    # colors = cmap_PuBuGn(np.linspace(0, 1, NVARS))

    for ii, ax in enumerate(axs.flatten()):
        # Plot the median 
        handles=[]
        for vv, var in enumerate(parvars): 
            hh, = ax.plot(xx, np.median(yy_bg[vv][ii], axis=-1), color=colors[var], lw=3, linestyle='--')
            # for aa, nn in enumerate(idx):
            #     ax.plot(xx, yy_bg[ii][:,nn], linestyle='-', alpha=0.75, color=colors[aa])  
            handles.append(hh)


        for vv, var in enumerate(parvars):
            # for aa, nn in enumerate(idx):
                # for ll in range(3):
                # edgecolor = 'k' if ll==0 else None
            ymed = np.median(yy_ss[vv][ii], axis=-1)
            ymax = np.max(yy_ss[vv][ii], axis=-1)
            ymin = np.min(yy_ss[vv][ii], axis=-1)
            if ii == 2:
                ax.errorbar(xx, ymin, yerr=(ymin-ymin, ymed-ymin), color=colors[var], alpha=0.5, 
                        capsize=2, lolims=True, marker='o', markersize=3, linestyle='')
            else:
                ax.errorbar(xx, ymax, yerr=(ymax-ymed, ymax-ymax), color=colors[var], alpha=0.5, 
                        capsize=2, uplims=True, marker='o', markersize=3, linestyle='')
            # ax.scatter(xx, ymed, marker='o', color=colors[vv], alpha=0.5, s=20)
            
        for vv, var in enumerate(parvars):
            # Plot the loudest single sources confidence intervals
            for pp in [68,]:
                percs = pp / 2
                percs = [50 - percs, 50 + percs]
                ax.fill_between(xx, *np.percentile(yy_ss[vv][ii], percs, axis=-1), alpha=0.25, color=colors[var])
        

        # label axes
        # if ii>=3:
        ax.set_ylabel(ylabels[ii])

    axs[-1].set_xlabel(plot.LABEL_GW_FREQUENCY_YR)
    plt.subplots_adjust(wspace=0, hspace=0)
    plot._twin_hz(axs[0], nano=True)

    # fig.tight_layout()
    labels = []
    for vv, var in enumerate(parvars):
        labels.append(f"{params[var][TARGET]:.2f}")

    fig.legend(handles=handles, labels=labels, bbox_to_anchor=(0.5,0.02), loc='lower center', ncols=len(parvars), title=plot.PARAM_KEYS[TARGET])
    fig.savefig(f'/Users/emigardiner/GWs/holodeck/output/figures/bigplots/hcprop_var/hcprop_vs_freq_var5_{TARGET}_uplims_68ci.png',
                dpi=300)
    return fig
fig = plot_uplims(xx, yy_ss, yy_bg, colors = cmap_Greens(np.linspace(0, 1, NVARS)))

In [ ]:
ylabels = ['Char. Strain', 'Mass [M$_\odot$]',# 'Mass Ratio', 
        'Distance [Mpc]',]

fig, axs = plot.figax_single(nrows=3, ncols=1, sharex=True, height=10)
xx = fobs_cents*YR
nsamp = 5

colors = cmap_Greens(np.linspace(0, 1, NVARS))
# colors = cmap_PuBuGn(np.linspace(0, 1, NVARS))

for ii, ax in enumerate(axs.flatten()):
    # Plot the median 
    handles=[]
    for vv, var in enumerate(parvars): 
        hh, = ax.plot(xx, np.median(yy_bg[vv][ii], axis=-1), color=colors[var], lw=3)
        # for aa, nn in enumerate(idx):
        #     ax.plot(xx, yy_bg[ii][:,nn], linestyle='-', alpha=0.75, color=colors[aa])  
        handles.append(hh)

    for vv, var in enumerate(parvars):
        xx_ss = np.repeat(xx, NREALS).reshape(NFREQS,NREALS)
        ax.scatter(xx_ss, yy_ss[vv][ii], marker='o', color=colors[var], alpha=0.1, s=5)


    # for vv, var in enumerate(parvars):
    #     # Plot the loudest single sources confidence intervals
    #     for pp in [95,]:
    #         percs = pp / 2
    #         percs = [50 - percs, 50 + percs]
    #         ax.fill_between(xx, *np.percentile(yy_ss[vv][ii], percs, axis=-1), alpha=0.25, color=colors[var])


    # label axes
    # if ii>=3:
    ax.set_ylabel(ylabels[ii])

axs[-1].set_xlabel(plot.LABEL_GW_FREQUENCY_YR)
plt.subplots_adjust(wspace=0, hspace=0)
plot._twin_hz(axs[0], nano=True)

# fig.tight_layout()
labels = []
for vv, var in enumerate(parvars):
    labels.append(f"{params[var][TARGET]:.2f}")

fig.legend(handles=handles, labels=labels, bbox_to_anchor=(0.5,0.02), loc='lower center', ncols=len(parvars), title=plot.PARAM_KEYS[TARGET])
fig.savefig(f'/Users/emigardiner/GWs/holodeck/output/figures/bigplots/hcprop_var/hcprop_vs_freq_var5_{TARGET}_scatter.png',
            dpi=300)

# Plots From Functions

## hard_time

In [ ]:
TARGET = 'hard_time'
NVARS = 21
NREALS = 500

# parvars = [0,5,10,15,20]
parvars = [0,10,20]
yy_ss = []
yy_bg = []
data, params = get_data(TARGET, nvars=NVARS, nreals=NREALS,
                        path='/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
print(data.shape)
for vv, var in enumerate(parvars):
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']

    sspar = data[var]['sspar']
    bgpar = data[var]['bgpar']

    sspar = sings.all_sspars(fobs_cents, sspar)
    bgpar = bgpar*sings.par_units[:,np.newaxis,np.newaxis]
    sspar = sspar*sings.par_units[:,np.newaxis,np.newaxis,np.newaxis]


   # parameters to plot
    _yy_ss = [hc_ss[...,0], sspar[0,...,0], #sspar[1,...,0], # sspar[2,],  # strain, mass, mass ratio,
            sspar[4,...,0]] # final comoving distance, single loudest only

    _yy_bg = [hc_bg, bgpar[0], #bgpar[1],  # strain, mass, mass ratio, initial redshift, final com distance
            bgpar[4],]
    yy_ss.append(_yy_ss)
    yy_bg.append(_yy_bg)

fig1 = plot_95ci(xx, yy_ss, yy_bg, colors = cmap_Blues(np.linspace(0, 1, NVARS)))
fig2 = plot_errbars(xx, yy_ss, yy_bg, colors = cmap_Blues(np.linspace(0, 1, NVARS)))
fig3 = plot_uplims(xx, yy_ss, yy_bg, colors = cmap_Blues(np.linspace(0, 1, NVARS)))

## GSMF_phi0, 5vars

In [ ]:
TARGET = 'gsmf_mchar0_log10'
NVARS = 21
NREALS = 500

parvars = [0,5,10,15,20]
yy_ss = []
yy_bg = []
data, params = get_data(TARGET, nvars=NVARS, nreals=NREALS,
                        path='/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
print(data.shape)
for vv, var in enumerate(parvars):
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']

    sspar = data[var]['sspar']
    bgpar = data[var]['bgpar']

    sspar = sings.all_sspars(fobs_cents, sspar)
    bgpar = bgpar*sings.par_units[:,np.newaxis,np.newaxis]
    sspar = sspar*sings.par_units[:,np.newaxis,np.newaxis,np.newaxis]


   # parameters to plot
    _yy_ss = [hc_ss[...,0], sspar[0,...,0], #sspar[1,...,0], # sspar[2,],  # strain, mass, mass ratio,
            sspar[4,...,0]] # final comoving distance, single loudest only

    _yy_bg = [hc_bg, bgpar[0], #bgpar[1],  # strain, mass, mass ratio, initial redshift, final com distance
            bgpar[4],]
    yy_ss.append(_yy_ss)
    yy_bg.append(_yy_bg)

fig1 = plot_95ci(xx, yy_ss, yy_bg, colors = cmap_Greens(np.linspace(0, 1, NVARS)))
fig2 = plot_errbars(xx, yy_ss, yy_bg, colors = cmap_Greens(np.linspace(0, 1, NVARS)))
fig3 = plot_uplims(xx, yy_ss, yy_bg, colors = cmap_Greens(np.linspace(0, 1, NVARS)))

## GSMF_mchar0_log10, 5vars

In [ ]:
TARGET = 'gsmf_mchar0_log10'
NVARS = 21
NREALS = 500

parvars = [0,5,10,15,20]
yy_ss = []
yy_bg = []
data, params = get_data(TARGET, nvars=NVARS, nreals=NREALS,
                        path='/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
print(data.shape)
for vv, var in enumerate(parvars):
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']

    sspar = data[var]['sspar']
    bgpar = data[var]['bgpar']

    sspar = sings.all_sspars(fobs_cents, sspar)
    bgpar = bgpar*sings.par_units[:,np.newaxis,np.newaxis]
    sspar = sspar*sings.par_units[:,np.newaxis,np.newaxis,np.newaxis]


   # parameters to plot
    _yy_ss = [hc_ss[...,0], sspar[0,...,0], #sspar[1,...,0], # sspar[2,],  # strain, mass, mass ratio,
            sspar[4,...,0]] # final comoving distance, single loudest only

    _yy_bg = [hc_bg, bgpar[0], #bgpar[1],  # strain, mass, mass ratio, initial redshift, final com distance
            bgpar[4],]
    yy_ss.append(_yy_ss)
    yy_bg.append(_yy_bg)


fig1 = plot_95ci(xx, yy_ss, yy_bg, colors = cmap_Greens(np.linspace(0, 1, NVARS)))
fig2 = plot_errbars(xx, yy_ss, yy_bg, colors = cmap_Greens(np.linspace(0, 1, NVARS)))
fig3 = plot_uplims(xx, yy_ss, yy_bg, colors = cmap_Greens(np.linspace(0, 1, NVARS)))

## mmb_mamp_log10, 5vars

In [ ]:
TARGET = 'mmb_mamp_log10'
NVARS = 21
NREALS = 500

parvars = [0,5,10,15,20]
yy_ss = []
yy_bg = []
data, params = get_data(TARGET, nvars=NVARS, nreals=NREALS,
                        path='/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
print(data.shape)
for vv, var in enumerate(parvars):
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']

    sspar = data[var]['sspar']
    bgpar = data[var]['bgpar']

    sspar = sings.all_sspars(fobs_cents, sspar)
    bgpar = bgpar*sings.par_units[:,np.newaxis,np.newaxis]
    sspar = sspar*sings.par_units[:,np.newaxis,np.newaxis,np.newaxis]


   # parameters to plot
    _yy_ss = [hc_ss[...,0], sspar[0,...,0], #sspar[1,...,0], # sspar[2,],  # strain, mass, mass ratio,
            sspar[4,...,0]] # final comoving distance, single loudest only

    _yy_bg = [hc_bg, bgpar[0], #bgpar[1],  # strain, mass, mass ratio, initial redshift, final com distance
            bgpar[4],]
    yy_ss.append(_yy_ss)
    yy_bg.append(_yy_bg)


fig1 = plot_95ci(xx, yy_ss, yy_bg, colors = cmap_Oranges(np.linspace(0, 1, NVARS)))
fig2 = plot_errbars(xx, yy_ss, yy_bg, colors = cmap_Oranges(np.linspace(0, 1, NVARS)))
fig3 = plot_uplims(xx, yy_ss, yy_bg, colors = cmap_Oranges(np.linspace(0, 1, NVARS)))

# mmb_scatter_dex

In [ ]:
TARGET = 'mmb_scatter_dex'
NVARS = 21
NREALS = 500

parvars = [0,5,10,15,20]
yy_ss = []
yy_bg = []
data, params = get_data(TARGET, nvars=NVARS, nreals=NREALS,
                        path='/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
print(data.shape)
for vv, var in enumerate(parvars):
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']

    sspar = data[var]['sspar']
    bgpar = data[var]['bgpar']

    sspar = sings.all_sspars(fobs_cents, sspar)
    bgpar = bgpar*sings.par_units[:,np.newaxis,np.newaxis]
    sspar = sspar*sings.par_units[:,np.newaxis,np.newaxis,np.newaxis]


   # parameters to plot
    _yy_ss = [hc_ss[...,0], sspar[0,...,0], #sspar[1,...,0], # sspar[2,],  # strain, mass, mass ratio,
            sspar[4,...,0]] # final comoving distance, single loudest only

    _yy_bg = [hc_bg, bgpar[0], #bgpar[1],  # strain, mass, mass ratio, initial redshift, final com distance
            bgpar[4],]
    yy_ss.append(_yy_ss)
    yy_bg.append(_yy_bg)


fig1 = plot_95ci(xx, yy_ss, yy_bg, colors = cmap_Oranges(np.linspace(0, 1, NVARS)))
fig2 = plot_errbars(xx, yy_ss, yy_bg, colors = cmap_Oranges(np.linspace(0, 1, NVARS)))
fig3 = plot_uplims(xx, yy_ss, yy_bg, colors = cmap_Oranges(np.linspace(0, 1, NVARS)))

# hard_gamma_inner

In [ ]:
TARGET = 'hard_gamma_inner'
NVARS = 21
NREALS = 500

parvars = [0,5,10,15,20]
yy_ss = []
yy_bg = []
data, params = get_data(TARGET, nvars=NVARS, nreals=NREALS,
                        path='/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
print(data.shape)
for vv, var in enumerate(parvars):
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']

    sspar = data[var]['sspar']
    bgpar = data[var]['bgpar']

    sspar = sings.all_sspars(fobs_cents, sspar)
    bgpar = bgpar*sings.par_units[:,np.newaxis,np.newaxis]
    sspar = sspar*sings.par_units[:,np.newaxis,np.newaxis,np.newaxis]


   # parameters to plot
    _yy_ss = [hc_ss[...,0], sspar[0,...,0], #sspar[1,...,0], # sspar[2,],  # strain, mass, mass ratio,
            sspar[4,...,0]] # final comoving distance, single loudest only

    _yy_bg = [hc_bg, bgpar[0], #bgpar[1],  # strain, mass, mass ratio, initial redshift, final com distance
            bgpar[4],]
    yy_ss.append(_yy_ss)
    yy_bg.append(_yy_bg)


fig1 = plot_95ci(xx, yy_ss, yy_bg, colors = cmap_Blues(np.linspace(0, 1, NVARS)))
fig2 = plot_errbars(xx, yy_ss, yy_bg, colors = cmap_Blues(np.linspace(0, 1, NVARS)))
fig3 = plot_uplims(xx, yy_ss, yy_bg, colors = cmap_Blues(np.linspace(0, 1, NVARS)))

# The charstrain plot to end all charstrain plots

In [ ]:
parvars = [0,1,2]
hc_ss_all = []
hc_bg_all= []
text = []
TARGET = 'gsmf_phi0'
data, params = get_data(TARGET, nvars=3, nreals=500, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
for vv, var in enumerate(parvars):    
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']
    hc_ss_all.append(hc_ss)
    hc_bg_all.append(hc_bg)
    text.append("$\psi_0$=%.2f" % params[var][TARGET])

hc_ss_phi0 = hc_ss_all
hc_bg_phi0 = hc_bg_all
text_phi0 = text

In [ ]:
parvars = [0,1,2]
hc_ss_all = []
hc_bg_all= []
text = []
TARGET = 'gsmf_mchar0_log10'
data, params = get_data(TARGET, nvars=3, nreals=500, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
for vv, var in enumerate(parvars):    
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']
    hc_ss_all.append(hc_ss)
    hc_bg_all.append(hc_bg)
    text.append("log($M_{\psi,0}$/M$_\odot$)=%.2f" % params[var][TARGET])

hc_ss_mchar0 = hc_ss_all
hc_bg_mchar0 = hc_bg_all
text_mchar0 = text

In [ ]:
parvars = [0,10,20]
hc_ss_all = []
hc_bg_all= []
text = []
TARGET = 'mmb_mamp_log10'
data, params = get_data(TARGET, nvars=21, nreals=500, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
for vv, var in enumerate(parvars):    
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']
    hc_ss_all.append(hc_ss)
    hc_bg_all.append(hc_bg)
    text.append("log($\mu$/M$_\odot$)=%.2f" % params[var][TARGET])

hc_ss_mamp = hc_ss_all
hc_bg_mamp = hc_bg_all
text_mamp = text

In [ ]:
parvars = [0,10,20]
hc_ss_all = []
hc_bg_all= []
text = []
TARGET = 'mmb_scatter_dex'
data, params = get_data(TARGET, nvars=21, nreals=500, nskies=100, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_09B')
for vv, var in enumerate(parvars):    
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']
    hc_ss_all.append(hc_ss)
    hc_bg_all.append(hc_bg)
    text.append("$\epsilon_\mu$=%.2f dex" % params[var][TARGET])

hc_ss_scatter = hc_ss_all
hc_bg_scatter = hc_bg_all
text_scatter = text

In [ ]:
parvars = [0,10,20]
hc_ss_all = []
hc_bg_all = []
text = []
TARGET = 'hard_time'
data, params = get_data(TARGET, nvars=21, nreals=500, nskies=100, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz')
for vv, var in enumerate(parvars):    
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']
    hc_ss_all.append(hc_ss)
    hc_bg_all.append(hc_bg)
    text.append("$\\tau_\mathrm{hard}=%.2f$" % params[var][TARGET])

hc_ss_hardtime = hc_ss_all
hc_bg_hardtime = hc_bg_all
text_hardtime = text

In [ ]:
parvars = [0,10,20]
hc_ss_all = []
hc_bg_all= []
text = []
TARGET = 'hard_gamma_inner'
data, params = get_data(TARGET, nvars=21, nreals=500, nskies=100, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_09B')
for vv, var in enumerate(parvars):    
    hc_ss = data[var]['hc_ss']
    hc_bg = data[var]['hc_bg']
    hc_ss_all.append(hc_ss)
    hc_bg_all.append(hc_bg)
    text.append("$\\nu_\mathrm{inner}$=%.2f" % params[var][TARGET])

hc_ss_gamma = hc_ss_all
hc_bg_gamma = hc_bg_all
text_gamma = text

In [ ]:
print(np.shares_memory(hc_ss_hardtime, hc_ss_all))

In [ ]:
nsamp = 5    # number of sample GWB spectra to plot
colors = [
    '#ff7f0f', # orange
    '#6a3d9a', # purple
    '#f0027f', # pink
    '#a6d853', # green
    '#15becf', # teal
]

target_colors = [
    '#336948',  '#336948', 
    '#9e5c41', '#9e5c41',
    '#2d839f', '#2d839f',
]

fig, axs = plot.figax_double(ncols=3, nrows = 6, sharex=True, sharey=True, height=14)
# axs[0].set_ylabel(plot.LABEL_CHARACTERISTIC_STRAIN)
fig.text(0.04, 0.5, plot.LABEL_CHARACTERISTIC_STRAIN, ha='center', va='bottom', rotation='vertical')
axs[-1, 1].set_xlabel(plot.LABEL_GW_FREQUENCY_YR)
plt.subplots_adjust(wspace=0, hspace=0)

# plot as a function of frequency /yr
xx = fobs_cents * YR
xx_ss = np.repeat(xx, NLOUDEST).reshape(NFREQS, NLOUDEST)
# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)


targets = ['gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_time', 'hard_gamma_inner']
hc_ss_targets = [hc_ss_phi0, hc_ss_mchar0, hc_ss_mamp, hc_ss_scatter, hc_ss_hardtime, hc_ss_gamma]
hc_bg_targets = [hc_bg_phi0, hc_bg_mchar0, hc_bg_mamp, hc_bg_scatter, hc_bg_hardtime, hc_bg_gamma]
text_targets = [text_phi0, text_mchar0, text_mamp, text_scatter, text_hardtime, text_gamma]
for rr, ax_row in enumerate(axs):
    text = text_targets[rr]
    hc_ss_all = hc_ss_targets[rr]
    hc_bg_all = hc_bg_targets[rr]
    for cc, ax in enumerate(ax_row):
        
        ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0)

        hc_ss = hc_ss_all[cc]
        hc_bg = hc_bg_all[cc]

        # Plot the median GWB spectrum
        ax.plot(xx, np.median(hc_bg, axis=-1), 'k-')

        # Plot `nsamp` random spectra 
        seed = np.random.randint(99999)   # get a random number
        print(f"{ii=}, {seed=}")                       # print it out so we can reuse it if desired
        np.random.seed(seed)              # set the random seed

        nsamp = np.min([nsamp, NREALS])

        # select random realizations to plot
        idx = np.random.choice(NREALS, nsamp, replace=False)
        for ci, ii in enumerate(idx):

            for ll in range(5):
                edgecolor = 'k' if ll==0 else None
                ax.scatter(xx, hc_ss[:,ii,ll], color=colors[ci], alpha=0.3, edgecolor=edgecolor,
                        s=20)

        for ci, ii in enumerate(idx):
            ax.plot(xx, hc_bg[:,ii], linestyle='-', alpha=0.75, color=colors[ci])  

        # plot contours at 50% and 98% confidence intervals
        for pp in [50, 98]:
            percs = pp / 2
            percs = [50 - percs, 50 + percs]
            ax.fill_between(xx, *np.percentile(hc_bg, percs, axis=-1), alpha=0.25, color='k')
        ax.text(
            0.01,0.01, text[cc], transform=ax.transAxes, color=target_colors[rr],
            horizontalalignment='left', verticalalignment='bottom')
        
    # plt.show()
fig.savefig(f'/Users/emigardiner/GWs/holodeck/output/figures/bigplots/hc_3x6.png', dpi=300)